<a href="https://colab.research.google.com/github/ab17254/ce888/blob/main/Assignment/AIBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'aif360[all]'

     |████████████████████████████████| 184kB 5.8MB/s 
     |████████████████████████████████| 9.2MB 7.2MB/s 
     |████████████████████████████████| 972kB 43.0MB/s 
     |████████████████████████████████| 276kB 40.5MB/s 
     |████████████████████████████████| 2.6MB 33.4MB/s 
     |████████████████████████████████| 21.2MB 2.0MB/s 
     |████████████████████████████████| 358kB 43.0MB/s 
     |████████████████████████████████| 552kB 33.4MB/s 
     |████████████████████████████████| 235kB 43.4MB/s 
     |████████████████████████████████| 21.6MB 1.8MB/s 
     |████████████████████████████████| 3.4MB 38.0MB/s 
     |████████████████████████████████| 747kB 40.7MB/s 
     |████████████████████████████████| 25.3MB 44.9MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=327caf33a14745cae8a717f35c23ecd422e1cefbc3e1aadc6de548c5ab28fc1c
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
  Created wheel 

In [2]:
# Ensure data.zip is downloaded from github repository and uploaded to runtime
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: content/data/
  inflating: content/data/arrest_essex.csv  
  inflating: content/data/arrest_suffolk.csv  
  inflating: content/data/arrest_manchester.csv  
  inflating: content/data/arrest_lancashire.csv  
  inflating: content/data/arrest_london.csv  
  inflating: content/data/data_all.csv  
  inflating: content/data/arrest_all.csv  
  inflating: content/data/arrest_wiltshire.csv  
  inflating: content/data/arrest_all_new.csv  


In [3]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import ReweighingMeta
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, PostProcessingMeta
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

In [4]:
# Read data into dataframe
df = pd.read_csv('content/data/arrest_all_new.csv')

In [5]:
# Select a random sample of 10,000 items. Anything more increases runtime and sometime causes session to crash due to high RAM usage
df = df.sample(n=10000)

In [6]:
df['Time'] = df['Time'].str[:-3]

In [7]:
df_copy = df.copy()

In [8]:
# Creates an multi-index dataframe
df = df.set_index(['index','Ethnicity','Gender'])

In [10]:
df

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
93130,White Irish,Male,2011,Unknown,Northumbria,-
7366,White,Female,2008,18 - 20 years,Greater Manchester,1328
46265,Other,All,2012,10 - 17 years,Dorset,6
28870,Black,Female,2017,18 - 20 years,Humberside,-
40186,Unreported,All,2007,Under 10 years,Surrey,2
...,...,...,...,...,...,...
207453,Mixed White and Black African,All,2013,18 - 20 years,Durham,3
181679,Any other mixed/multiple ethnic background,All,2006,All,Warwickshire,29
95875,Black African,Female,2012,Under 10 years,London City of,-


In [11]:
X = df
y = df.iloc[:,3]

In [12]:
print(X.shape)
print(y.shape)

(10000, 4)
(10000,)


In [13]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)

In [14]:
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

In [ ]:
# X = pd.get_dummies(X)

In [15]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.50)

In [16]:
X_train

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
6627,12,2,2016,All,Northamptonshire,49
9063,0,1,2014,18 - 20 years,Essex,422
9693,14,1,2012,10 - 17 years,Nottinghamshire,1
5533,9,2,2016,21 years and over,Staffordshire,114
2063,19,0,2011,10 - 17 years,Northumbria,35
...,...,...,...,...,...,...
3025,7,2,2007,18 - 20 years,North Yorkshire,-
4141,5,2,2011,10 - 17 years,Norfolk,70
405,17,2,2009,21 years and over,Cleveland,87


In [17]:
X_test

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
1149,20,2,2015,Unknown,Nottinghamshire,2
184,17,2,2007,Under 10 years,Gwent,-
398,17,1,2009,Unknown,Metropolitan Police,3
7113,22,0,2008,Unknown,Merseyside,-
1287,8,1,2017,10 - 17 years,London City of,4
...,...,...,...,...,...,...
8766,0,2,2013,All,Dyfed-Powys,7544
2740,7,1,2006,Under 10 years,Northamptonshire,-
8557,11,0,2014,All,Merseyside,39


In [18]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(X_train)
X_train  = pd.DataFrame(ohe.transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), index=X_test.index)

In [19]:
y_train.head()

index  Ethnicity  Gender
6627   12         2         1070
9063   0          1          975
9693   14         1            2
5533   9          2           92
2063   19         0          854
dtype: int64

In [20]:
X_train

,,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002
index,Ethnicity,Gender,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6627,12,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9063,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9693,14,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5533,9,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2063,19,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,7,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4141,5,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
405,17,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Initial run of algorithm
y_pred = LogisticRegression(solver='lbfgs').fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

0.809

In [22]:
disparate_impact_ratio(y_test, y_pred, prot_attr='Ethnicity')

0.6675623227347365

In [23]:
# Debiased and hypertuned run of algorithm
rew = ReweighingMeta(estimator=LogisticRegression(solver='lbfgs'))

params = {'estimator__C': [1, 10], 'reweigher__prot_attr': ['Ethnicity']}

clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


0.8038
{'estimator__C': 10, 'reweigher__prot_attr': 'Ethnicity'}


In [24]:
 disparate_impact_ratio(y_test, clf.predict(X_test), prot_attr='Ethnicity')


0.6868189281982386